In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
from joblib import Parallel, delayed
import os
import multiprocessing
import re

# B004

In [4]:
# For Luis
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/B004/konbert-export-6426ee9112484.csv")
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/B004/konbert-export-cb72e3996b7f4.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [5]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [6]:
ozone_data

,index,Samplingpoint,Pollutant,Start,End,Value,Unit,AggType,Validity,Verification,ResultTime,DataCapture,FkObservationLog
0,0,BE/SPO-BETB004_00007_100,7,2001-01-01T00:00:00,2001-01-01T01:00:00,30.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
1,1,BE/SPO-BETB004_00007_100,7,2001-01-01T01:00:00,2001-01-01T02:00:00,28.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
2,2,BE/SPO-BETB004_00007_100,7,2001-01-01T02:00:00,2001-01-01T03:00:00,29.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
3,3,BE/SPO-BETB004_00007_100,7,2001-01-01T03:00:00,2001-01-01T04:00:00,31.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
4,4,BE/SPO-BETB004_00007_100,7,2001-01-01T04:00:00,2001-01-01T05:00:00,33.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
183931,87643,BE/SPO-BETB004_00007_100,7,2022-12-31T20:00:00,2022-12-31T21:00:00,39.0,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,03d4804a-8f49-43f1-9f3b-88a4568e49e1
183932,87644,BE/SPO-BETB004_00007_100,7,2022-12-31T21:00:00,2022-12-31T22:00:00,38.0,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,03d4804a-8f49-43f1-9f3b-88a4568e49e1
183933,87645,BE/SPO-BETB004_00007_100,7,2022-12-31T22:00:00,2022-12-31T23:00:00,40.5,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,03d4804a-8f49-43f1-9f3b-88a4568e49e1
183934,87646,BE/SPO-BETB004_00007_100,7,2022-12-31T23:00:00,2023-01-01T00:00:00,42.5,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,03d4804a-8f49-43f1-9f3b-88a4568e49e1


In [7]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [8]:
s = data.shape[0]

In [9]:
data

,Start,Concentration
0,2008-01-01 00:00:00,NaN
1,2008-01-01 01:00:00,NaN
2,2008-01-01 02:00:00,NaN
3,2008-01-01 03:00:00,NaN
4,2008-01-01 04:00:00,NaN
...,...,...
105042,2019-12-31 19:00:00,2.0
105043,2019-12-31 20:00:00,2.0
105044,2019-12-31 21:00:00,2.0
105045,2019-12-31 22:00:00,2.0


In [10]:
data['Concentration'].describe()

count    100186.000000
mean         35.295071
std          26.467143
min           2.000000
25%          13.500000
50%          32.000000
75%          51.000000
max         246.000000
Name: Concentration, dtype: float64

In [11]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

95.37254752634536


In [12]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [13]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [14]:
data['Concentration'].describe()

count    4252.000000
mean       50.307414
std        28.504139
min         2.000000
25%        29.812500
50%        47.687500
75%        66.515625
max       198.687500
Name: Concentration, dtype: float64

In [15]:
s = data.shape[0]

In [16]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

97.1441626684944


# B006

In [18]:
# For Luis
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/B006/konbert-export-7fd762f5a1454.csv")
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/B006/konbert-export-8a14ae4f0c844.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [19]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [20]:
ozone_data

,index,Samplingpoint,Pollutant,Start,End,Value,Unit,AggType,Validity,Verification,ResultTime,DataCapture,FkObservationLog
0,0,BE/SPO-BETB006_00007_100,7,2002-01-01T00:00:00,2002-01-01T01:00:00,4.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
1,1,BE/SPO-BETB006_00007_100,7,2002-01-01T01:00:00,2002-01-01T02:00:00,5.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
2,2,BE/SPO-BETB006_00007_100,7,2002-01-01T02:00:00,2002-01-01T03:00:00,3.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
3,3,BE/SPO-BETB006_00007_100,7,2002-01-01T03:00:00,2002-01-01T04:00:00,4.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
4,4,BE/SPO-BETB006_00007_100,7,2002-01-01T04:00:00,2002-01-01T05:00:00,3.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131395,35059,BE/SPO-BETB006_00007_100,7,2016-12-31T20:00:00,2016-12-31T21:00:00,NaN,ug.m-3,hour,-1,1,2017-09-27T08:09:16,0.0,69cae6e0-fee4-4b75-ac3e-64b20180c7de
131396,35060,BE/SPO-BETB006_00007_100,7,2016-12-31T21:00:00,2016-12-31T22:00:00,NaN,ug.m-3,hour,-1,1,2017-09-27T08:09:16,0.0,69cae6e0-fee4-4b75-ac3e-64b20180c7de
131397,35061,BE/SPO-BETB006_00007_100,7,2016-12-31T22:00:00,2016-12-31T23:00:00,NaN,ug.m-3,hour,-1,1,2017-09-27T08:09:16,0.0,69cae6e0-fee4-4b75-ac3e-64b20180c7de
131398,35062,BE/SPO-BETB006_00007_100,7,2016-12-31T23:00:00,2017-01-01T00:00:00,NaN,ug.m-3,hour,-1,1,2017-09-27T08:09:16,0.0,69cae6e0-fee4-4b75-ac3e-64b20180c7de


In [21]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [22]:
s = data.shape[0]

In [23]:
data['Concentration'].describe()

count    77911.000000
mean        40.605678
std         27.510324
min          1.500000
25%         18.000000
50%         39.500000
75%         58.000000
max        212.500000
Name: Concentration, dtype: float64

In [24]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

88.96387138028685


In [25]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [26]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [27]:
data['Concentration'].describe()

count    3317.000000
mean       54.571770
std        28.577214
min         2.000000
25%        34.937500
50%        53.062500
75%        71.625000
max       182.562500
Name: Concentration, dtype: float64

In [28]:
s = data.shape[0]

In [29]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

90.87671232876713


# R001

In [30]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R001/konbert-export-ad0893d9b8bb4.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R001/konbert-export-d4067519df294.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [31]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [33]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [34]:
data

,Start,Concentration
0,2007-01-01 00:00:00,55.0
1,2007-01-01 01:00:00,68.0
2,2007-01-01 02:00:00,67.0
3,2007-01-01 03:00:00,64.0
4,2007-01-01 04:00:00,69.0
...,...,...
113754,2019-12-31 19:00:00,2.5
113755,2019-12-31 20:00:00,3.0
113756,2019-12-31 21:00:00,2.0
113757,2019-12-31 22:00:00,2.5


In [35]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [36]:
data['Concentration'].describe()

count    110231.000000
mean         35.472775
std          25.986265
min           1.500000
25%          13.000000
50%          33.500000
75%          52.000000
max         209.000000
Name: Concentration, dtype: float64

In [37]:
s = data.shape[0]

In [38]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

96.89870691549679


In [39]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [40]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [41]:
s = data.shape[0]

In [42]:
data['Concentration'].describe()

count    4688.000000
mean       49.999933
std        27.065355
min         2.000000
25%        30.562500
50%        48.531250
75%        66.125000
max       183.187500
Name: Concentration, dtype: float64

In [43]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

98.90295358649789


# R201

In [44]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R201/konbert-export-c8e1bbfbcde24.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R201/konbert-export-cc632b75de554.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

<ipython-input-44-20a5c86ccec5>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R201/konbert-export-c8e1bbfbcde24.csv")


In [45]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [46]:
ozone_data

,index,Samplingpoint,Pollutant,Start,End,Value,Unit,AggType,Validity,Verification,ResultTime,DataCapture,FkObservationLog
0,0,BE/SPO-BETR201_00007_100,7,1994-01-01T00:00:00,1994-01-01T01:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
1,1,BE/SPO-BETR201_00007_100,7,1994-01-01T01:00:00,1994-01-01T02:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
2,2,BE/SPO-BETR201_00007_100,7,1994-01-01T02:00:00,1994-01-01T03:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
3,3,BE/SPO-BETR201_00007_100,7,1994-01-01T03:00:00,1994-01-01T04:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
4,4,BE/SPO-BETR201_00007_100,7,1994-01-01T04:00:00,1994-01-01T05:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157339,8755,BE/SPO-BETR201_00007_100,7,2013-12-31T20:00:00,2013-12-31T21:00:00,NaN,ug.m-3,hour,-1,1,2014-06-20T09:06:34,0.0,24afb3a4-0efd-4ed9-80e6-bf6786cd2f0a
157340,8756,BE/SPO-BETR201_00007_100,7,2013-12-31T21:00:00,2013-12-31T22:00:00,NaN,ug.m-3,hour,-1,1,2014-06-20T09:06:34,0.0,24afb3a4-0efd-4ed9-80e6-bf6786cd2f0a
157341,8757,BE/SPO-BETR201_00007_100,7,2013-12-31T22:00:00,2013-12-31T23:00:00,NaN,ug.m-3,hour,-1,1,2014-06-20T09:06:34,0.0,24afb3a4-0efd-4ed9-80e6-bf6786cd2f0a
157342,8758,BE/SPO-BETR201_00007_100,7,2013-12-31T23:00:00,2014-01-01T00:00:00,NaN,ug.m-3,hour,-1,1,2014-06-20T09:06:34,0.0,24afb3a4-0efd-4ed9-80e6-bf6786cd2f0a


In [47]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [48]:
data

,Start,Concentration
0,2007-01-01 00:00:00,55.0
1,2007-01-01 01:00:00,54.0
2,2007-01-01 02:00:00,71.0
3,2007-01-01 03:00:00,53.0
4,2007-01-01 04:00:00,55.0
...,...,...
60931,2013-12-31 20:00:00,NaN
60932,2013-12-31 21:00:00,NaN
60933,2013-12-31 22:00:00,NaN
60934,2013-12-31 23:00:00,NaN


In [49]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [50]:
data['Concentration'].describe()

count     49244.0
unique      354.0
top           2.0
freq       2062.0
Name: Concentration, dtype: float64

In [51]:
s = data.shape[0]

In [52]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

80.81265590127347


In [53]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [54]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [55]:
s = data.shape[0]

In [56]:
data['Concentration'].describe()

count    2121.000000
mean       54.602900
std        29.140808
min         0.000000
25%        34.500000
50%        53.125000
75%        72.437500
max       184.312500
Name: Concentration, dtype: float64

In [57]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

83.50393700787401


# R701

In [58]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R701/konbert-export-2a4a8517b0ef4.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R701/konbert-export-ae1f065d91d44.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [59]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [60]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [61]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [62]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [63]:
data['Concentration'].describe()

count    104115.000000
mean         39.421044
std          27.899030
min           0.000000
25%          17.000000
50%          38.000000
75%          57.000000
max         218.000000
Name: Concentration, dtype: float64

In [64]:
s = data.shape[0]

In [65]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

92.14450708463507


In [66]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [67]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [68]:
s = data.shape[0]

In [69]:
data['Concentration'].describe()

count    4516.000000
mean       55.010643
std        28.758760
min         0.312500
25%        35.875000
50%        54.468750
75%        71.437500
max       178.687500
Name: Concentration, dtype: float64

In [70]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

95.92183517417162


# R801

In [71]:
# For Luis
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R801/konbert-export-33bca227c02d4.csv")
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R801/konbert-export-9014afb433b24.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [72]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [73]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [74]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [75]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [76]:
data['Concentration'].describe()

count    104682.000000
mean         31.698243
std          26.205569
min           0.000000
25%           8.500000
50%          29.000000
75%          48.000000
max         241.000000
Name: Concentration, dtype: float64

In [77]:
s = data.shape[0]

In [78]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

92.92263991833474


In [79]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [80]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [81]:
s = data.shape[0]

In [82]:
data['Concentration'].describe()

count    4518.000000
mean       45.658228
std        27.733991
min         0.000000
25%        25.187500
50%        44.593750
75%        61.875000
max       190.500000
Name: Concentration, dtype: float64

In [83]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

96.25053259480187
